# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846909947467                   -0.70    4.8         
  2   -7.852329966916       -2.27       -1.53    1.0   17.3ms
  3   -7.852617845450       -3.54       -2.56    1.5   19.0ms
  4   -7.852646062216       -4.55       -2.92    2.5   22.5ms
  5   -7.852646545907       -6.32       -3.25    1.0   17.2ms
  6   -7.852646680978       -6.87       -4.22    1.2   18.0ms
  7   -7.852646686697       -8.24       -5.36    2.2   22.7ms
  8   -7.852646686727      -10.52       -5.63    1.8   20.4ms
  9   -7.852646686728      -12.01       -5.67    1.5   19.5ms
 10   -7.852646686730      -11.84       -6.85    1.0   17.9ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846864483179                   -0.70           4.8         
  2   -7.852553599665       -2.24       -1.63   0.80    2.0    212ms
  3   -7.852638951769       -4.07       -2.75   0.80    1.0   16.4ms
  4   -7.852646506449       -5.12       -3.32   0.80    1.8   19.4ms
  5   -7.852646677317       -6.77       -4.22   0.80    1.2   17.6ms
  6   -7.852646686601       -8.03       -4.76   0.80    2.2   21.1ms
  7   -7.852646686721       -9.92       -5.65   0.80    1.2   17.4ms
  8   -7.852646686730      -11.06       -6.51   0.80    2.0   19.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.380777e+01     3.580388e+00
 * time: 0.04623007774353027
     1     1.334017e+00     2.064945e+00
 * time: 0.22191905975341797
     2    -1.654318e+00     1.982039e+00
 * time: 0.2399120330810547
     3    -3.746801e+00     1.936985e+00
 * time: 0.26590704917907715
     4    -5.210869e+00     1.583788e+00
 * time: 0.2918729782104492
     5    -6.786747e+00     9.181488e-01
 * time: 0.3178119659423828
     6    -7.462761e+00     3.393644e-01
 * time: 0.3436310291290283
     7    -7.692958e+00     1.563139e-01
 * time: 0.36150217056274414
     8    -7.775823e+00     1.382081e-01
 * time: 0.37938499450683594
     9    -7.816763e+00     6.199076e-02
 * time: 0.3973960876464844
    10    -7.839437e+00     5.428677e-02
 * time: 0.41533398628234863
    11    -7.846859e+00     4.457109e-02
 * time: 0.4332301616668701
    12    -7.849840e+00     3.077597e-02
 * time: 0.451185941696167
    13    -7.851468e+00     2.070325e-02
 * time: 0.46936

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846751792369                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645860234                   -1.64         
  2   -7.852646686730       -6.08       -3.71    1.52s
  3   -7.852646686730      -13.15       -7.20    121ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.971271634438422e-7
|ρ_newton - ρ_scfv| = 8.260844754891872e-8
|ρ_newton - ρ_dm|   = 7.234545879182552e-10
